# IMPORTS

In [1]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle
from pathlib import Path
from datetime import timedelta
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

# SETUP

In [2]:
dir_tree_util_path = os.path.join("utils", "dir_tree.py")
exec(open(dir_tree_util_path).read())

# INPUTS

In [3]:
# Paths
path_input = PROJECT_DIRS["DADOS_VEQ_ANTT_DIR"]
path_output = PROJECT_DIRS["DADOS_DERIVADOS_DIR"]

In [4]:
periodo = list(range(2010, 2025))

# Criando um dataframe consolidado com os dados de tráfego

In [5]:
# arquivos dos dados
arquivos = os.listdir(path_input)

In [6]:
df_trafego = pd.DataFrame()
for ano in periodo:
    arquivo = [a for a in arquivos if a[:-4].endswith(str(ano))][0]
    df_ano = pd.read_csv(os.path.join(path_input, arquivo), sep=';', encoding='cp1252', low_memory=False)
    df_trafego = pd.concat([df_trafego,df_ano])

# EDA Inicial

In [7]:
df_trafego.head()

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
0,RODOVIA DO AÇO,01-01-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,44146,1,44146,NaN
1,RODOVIA DO AÇO,01-01-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,35771,1,35771,NaN
2,RODOVIA DO AÇO,01-02-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,33455,1,33455,NaN
3,RODOVIA DO AÇO,01-02-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,27109,1,27109,NaN
4,RODOVIA DO AÇO,01-03-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,31677,1,31677,NaN


In [8]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 10 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   concessionaria              object
 1   mes_ano                     object
 2   sentido                     object
 3   praca                       object
 4   categoria                   object
 5   tipo_de_veiculo             object
 6   volume_total                object
 7   multiplicador_de_tarifa     object
 8   volume_veiculo_equivalente  object
 9   tipo_de_cobranca            object
dtypes: object(10)
memory usage: 251.8+ MB


In [9]:
for col in df_trafego:
    print(f"{col}: {df_trafego[col].isna().sum()}")

concessionaria: 0
mes_ano: 0
sentido: 0
praca: 0
categoria: 0
tipo_de_veiculo: 0
volume_total: 344
multiplicador_de_tarifa: 0
volume_veiculo_equivalente: 0
tipo_de_cobranca: 74780


In [10]:
df_trafego[df_trafego['volume_total'].isna()].sample(9)

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
136458,RIOSP,01/10/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 6,Comercial,NaN,"4,00","0,00",N/I
130437,ECOVIAS DO CERRADO,01/08/2023,Crescente,"P7 - Jataí BR 364/GO km 156,550",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
126172,ECOSUL,01/12/2023,Decrescente,"Praça 04 BR-392/RS km 52,30",Categoria 9,Moto,NaN,"0,50","0,00",N/I
120975,ECO050,01/11/2023,Decrescente,"Praça 02 BR-050/GO km 226,00",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
134819,RIOSP,01/03/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 4,Comercial,NaN,"3,00","0,00",N/I
125442,ECOSUL,01/05/2023,Crescente,"Praça 03 BR-116/RS km 541,20",Categoria 9,Moto,NaN,"0,50","0,00",N/I
131074,ECOVIAS DO CERRADO,01/12/2023,Decrescente,P3 - ITUIUTABA,Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
130736,ECOVIAS DO CERRADO,01/10/2023,Decrescente,P4 - SANTA VITÓRIA,Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
121118,ECO050,01/12/2023,Crescente,"Praça 02 BR-050/GO km 226,00",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I


In [11]:
df_trafego[df_trafego['volume_total'].isna()]['volume_veiculo_equivalente'].unique()

array(['0,00'], dtype=object)

In [12]:
df_trafego['mes_ano'].str[:2].unique()

array(['01', '14', '28', '31', '30', '02', '03', '04', '05', '06', '07',
       '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27', '29'], dtype=object)

In [13]:
df_trafego['mes_ano'].str[3:5].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)

In [14]:
df_trafego['tipo_de_cobranca'].unique()

array([nan, 'N/I', 'Manual', 'Automática', 'Mista'], dtype=object)

In [15]:
df_trafego['volume_total'].isna().any()

np.True_

In [16]:
df_trafego['volume_total'].isna().sum()

np.int64(344)

In [17]:
df_trafego['multiplicador_de_tarifa'].unique()

array(['1', '2', '1,5', '3', '4', '5', '6', '0,5', '7', '8', '9', '10',
       1.0, 2.0, 1.5, 3.0, 4.0, 5.0, 6.0, 0.5, 7.0, 8.0, 9.0, 10.0,
       '2,00', '3,00', '4,00', '5,00', '6,00', '0,50', '1,00', '1,50',
       '7,00', '8,00', '9,00', '10,00', '0,00', '11,00', '13,00', '15,00',
       '12,00', '14,00', '18,00', '20,00', '16,00', '17,00', '19,00'],
      dtype=object)

# Tratamento dos dados

In [18]:
# Colunas de ano, mes e dia
df_trafego['year'] = df_trafego['mes_ano'].str[-4:].astype('int') 
df_trafego['month'] = df_trafego['mes_ano'].str[3:5].astype('int')
df_trafego['day'] = df_trafego['mes_ano'].str[:2].astype('int') 

In [19]:
# Recriando a coluna de mes_ano, agora como datetime ao invés de string. Tem que ser em ingles pq a função do pandas exige
df_trafego['mes_ano'] = pd.to_datetime(df_trafego[['year', 'month', 'day']]).dt.date

In [20]:
# preenchendo os NaNs, qdo aplicável:
df_trafego['volume_total'] = df_trafego['volume_total'].fillna(0) #o volume equivalente é zero sempre que volume_total é zero

In [21]:
# Convertendo os tipos de dados para numéricos:
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.split('.')[0] if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].astype('int')

df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].astype('float') # existe multiplicador de tarifa fracionário

In [22]:
# convertendo categoria para string (do contrário não salva para parquet):
df_trafego['categoria'] = df_trafego['categoria'].astype('string')

In [23]:
# convertendo o multiplicador de tarifa de string para decimal (p/manter a precisao):
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(Decimal)

In [24]:
# O tipo de tráfego ora está em maisúcula, ora em minúscula.
# Colocando tudo p/maiúscula
df_trafego['tipo_de_veiculo'] = df_trafego['tipo_de_veiculo'].str.upper()

In [25]:
# algumas concessionárias estão ora em maiúsculas ora em minúsculas
# colocando tudo para maiúscula
df_trafego['concessionaria'] = df_trafego['concessionaria'].str.upper()

In [26]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 13 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   concessionaria              object 
 1   mes_ano                     object 
 2   sentido                     object 
 3   praca                       object 
 4   categoria                   string 
 5   tipo_de_veiculo             object 
 6   volume_total                int64  
 7   multiplicador_de_tarifa     object 
 8   volume_veiculo_equivalente  float64
 9   tipo_de_cobranca            object 
 10  year                        int64  
 11  month                       int64  
 12  day                         int64  
dtypes: float64(1), int64(4), object(7), string(1)
memory usage: 320.5+ MB


In [27]:
df_trafego['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024])

In [28]:
df_trafego.query("year == 2024")['month'].unique()

array([1, 2, 3, 4, 5, 6])

## Criando coluna desambiguando categorias comercial x passeio

In [29]:
df_trafego['tipo_de_veiculo'].unique()

array(['PASSEIO', 'COMERCIAL', 'MOTO', 'VEÍCULO PEQUENO'], dtype=object)

In [30]:
df_trafego["TIPO_TRAFEGO"] = df_trafego['tipo_de_veiculo'].apply(lambda x: x if x == "COMERCIAL" else "PASSEIO")

In [31]:
df_trafego["TIPO_TRAFEGO"].unique()

array(['PASSEIO', 'COMERCIAL'], dtype=object)

## Criando colunas com trafégo em Veqs para comercial e passeio

In [32]:
df_trafego = df_trafego.rename(columns={'volume_veiculo_equivalente':'VEQS_TOTAL'})
df_trafego['VEQS_COMERCIAL'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'COMERCIAL',
    df_trafego['VEQS_TOTAL'],
    0
)
df_trafego['VEQS_PASSEIO'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'PASSEIO',
    df_trafego['VEQS_TOTAL'],
    0
)

# Criando dataframe com o período inicial e final dos dados por concessionária

In [33]:
df_periodos = df_trafego.groupby('concessionaria').agg(
    data_inicial=('mes_ano', 'min'),
    data_final=('mes_ano', 'max')
).reset_index()
df_periodos['data_inicial'] = pd.to_datetime(df_periodos['data_inicial']).dt.date
df_periodos['data_final'] = pd.to_datetime(df_periodos['data_final']).dt.date

In [34]:
df_periodos.index = df_periodos['concessionaria']
df_periodos = df_periodos.drop(columns=['concessionaria'])

In [35]:
df_periodos = df_periodos.sort_values(by='data_inicial', ascending=False)

# Criando dataframes agrupando VEQs por ANO/concessionaria/tipo de trafego

In [36]:
df_trafego.columns

Index(['concessionaria', 'mes_ano', 'sentido', 'praca', 'categoria',
       'tipo_de_veiculo', 'volume_total', 'multiplicador_de_tarifa',
       'VEQS_TOTAL', 'tipo_de_cobranca', 'year', 'month', 'day',
       'TIPO_TRAFEGO', 'VEQS_COMERCIAL', 'VEQS_PASSEIO'],
      dtype='object')

In [37]:
cols_veqs = ['VEQS_COMERCIAL','VEQS_PASSEIO']

In [38]:
dict_veqs_anual = {}
for col in cols_veqs:
    dict_veqs_anual[col] = df_trafego.pivot_table(
        index='concessionaria',
        columns='year',
        values=col,
        aggfunc='sum'
        ).T

In [39]:
df_periodos.iloc[0,0]

datetime.date(2024, 1, 1)

In [40]:
# retirando os anos incompletos
def filter_incomplete_years(df_trafego, df_periodos):
    df = df_trafego.copy()
    for conc in df_periodos.index:
        data_inicial = df_periodos.loc[conc,'data_inicial']
        data_final = df_periodos.loc[conc,'data_final']
        # print(data_inicial, data_final)
        years_drop = []
        if (data_inicial.day != 1) or (data_inicial.month != 1):
            years_drop.append(data_inicial.year)
        if (data_final.day != 31) or (data_final.month != 12):
            years_drop.append(data_final.year)
        # print(list(set(years_drop)))
        years_drop = list(set(years_drop))
        for year in years_drop:
            # print(df_trafego.loc[year, conc])
            df.loc[year, conc] = None
            # print(df_trafego.loc[year, conc])
    return df    

In [41]:
for veq, df in dict_veqs_anual.items():
    dict_veqs_anual[veq] = filter_incomplete_years(df, df_periodos)

# Criando dataframes agrupando VEQs por MÊS/concessionaria/tipo de trafego

In [42]:
# Criando uma coluna agrupando todos os registros em um mes ano para o final do mes para groupby
df_trafego['MES_ANO_f'] = pd.to_datetime(df_trafego['mes_ano'], format='%m/%Y') + pd.offsets.MonthEnd(0)
df_trafego['MES_ANO_f'] = df_trafego['MES_ANO_f'].dt.date

In [43]:
dict_veqs_mensal = {}
for col in cols_veqs:
    dict_veqs_mensal[col] = df_trafego.pivot_table(
        index='concessionaria',
        columns='MES_ANO_f',
        values=col,
        aggfunc='sum'
        ).T

In [44]:
def filter_incomplete_months(df_trafego, df_periodos):
    df = df_trafego.copy()

    for conc in df_periodos.index:
        data_inicial = df_periodos.loc[conc, 'data_inicial']
        data_final = df_periodos.loc[conc, 'data_final']
        
        # List to keep track of months to drop
        months_drop = []

        # Check if the starting month is incomplete
        if data_inicial.day != 1:
            # Drop the starting month if not starting from the 1st day
            incomplete_start = (data_inicial + pd.offsets.MonthEnd(0)).date()
            months_drop.append(incomplete_start)

        # Check if the ending month is incomplete
        if data_final != (data_final + pd.offsets.MonthEnd(0)):
            # Drop the ending month if not ending on the last day
            incomplete_end = (data_final + pd.offsets.MonthEnd(0)).date()
            months_drop.append(incomplete_end)

        # Remove duplicate dates (if any)
        months_drop = list(set(months_drop))

        # Remove the rows corresponding to incomplete months
        for month in months_drop:
            if month in df.index:
                df.loc[month, conc] = None
    return df    

In [45]:
for veq, df in dict_veqs_mensal.items():
    dict_veqs_mensal[veq] = filter_incomplete_months(df, df_periodos)

# Criando dataframes agrupando VEQs por TRIMESTRE/concessionaria/tipo de trafego

In [46]:
# TO-DO: retirar daqui, e colocar para agrupar após ter os dados dessazonalizados!

In [47]:
def group_by_tri(df_monthly):
    df_tri = df_monthly.copy()
    
    # Ensure the index is a datetime object
    df_tri.index = pd.to_datetime(df_tri.index)

    # Create a column for the quarter-year format
    df_tri['trimestre'] = df_tri.index.to_period('Q')
    
    # Identify quarters with any NaN values
    cols = [col for col in df_tri.columns if col != 'trimestre']
    for col in cols:
        # Find quarters where there is at least one NaN value in the column
        quarters_with_nan = df_tri.groupby('trimestre')[col].apply(lambda x: x.isna().any())
        
        # Filter only quarters that have NaN
        quarters_with_nan = quarters_with_nan[quarters_with_nan].index

        # Set all values in those quarters to zero for the column
        df_tri.loc[df_tri['trimestre'].isin(quarters_with_nan), col] = 0

    # Drop the helper column
    df_tri = df_tri.drop(columns=['trimestre'])

    # Group by trimester (quarter) and sum
    df_tri = df_tri.groupby(df_tri.index.to_period('Q')).sum()
    
    df_tri.index.name = 'trimestre'

    return df_tri


In [48]:
group_by_tri(dict_veqs_mensal['VEQS_COMERCIAL'])

concessionaria,AUTOPISTA FERNÃO DIAS,AUTOPISTA FLUMINENSE,AUTOPISTA LITORAL SUL,AUTOPISTA PLANALTO SUL,AUTOPISTA REGIS BITTENCOURT,CONCEBRA,CONCEPA,CONCER,CRO,CRT,ECO050,ECO101 CONCESSIONARIA DE RODOVIAS S/A,ECOPONTE,ECORIOMINAS,ECOSUL,ECOVIAS DO ARAGUAIA,ECOVIAS DO CERRADO,HOLDING DO SISTEMA RODOVIARIO RIO - SAO PAULO S.A.,MSVIA,NOVADUTRA,RIOSP,RODOVIA DO AÇO,TRANSBRASILIANA,VIA 040,VIA BAHIA,VIA BRASIL,VIA COSTEIRA,VIA SUL
trimestre,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010Q1,19118234.0,5541521.0,16068951.0,4647174.0,26099925.0,0.0,3441949.0,3010524.0,0.0,1992136.0,0.0,0.0,0.0,0.0,2992516.0,0.0,0.0,0.0,0.0,23693645.0,0.0,3081118.0,3423756.0,0.0,0.0,0.0,0.0,0.0
2010Q2,21643553.0,5701041.0,16910430.0,4987917.0,28377179.0,0.0,3380149.0,3157629.0,0.0,2015505.0,0.0,0.0,0.0,0.0,4325632.0,0.0,0.0,0.0,0.0,30087408.0,0.0,3342063.0,3786746.0,0.0,0.0,0.0,0.0,0.0
2010Q3,24157523.0,6183923.0,18332337.0,4948655.0,29787088.0,0.0,3510893.0,3418319.0,0.0,2221628.0,0.0,0.0,0.0,0.0,3666469.0,0.0,0.0,0.0,0.0,31267360.0,0.0,3246286.0,4151907.0,0.0,0.0,0.0,0.0,0.0
2010Q4,27296125.0,6452851.0,18619986.0,4971035.0,30620469.0,0.0,3763546.0,3523507.0,0.0,2334071.0,0.0,0.0,0.0,0.0,3442323.0,0.0,0.0,0.0,0.0,31332569.0,0.0,3217731.0,4152586.0,0.0,0.0,0.0,0.0,0.0
2011Q1,26160702.0,6215518.0,17250630.0,5212567.0,29285468.0,0.0,3540257.0,3533875.0,0.0,2188312.0,0.0,0.0,0.0,0.0,3626090.0,0.0,0.0,0.0,0.0,29355052.0,0.0,2984654.0,3786002.0,0.0,0.0,0.0,0.0,0.0
2011Q2,27159762.0,6435567.0,18224379.0,5126143.0,30475511.0,0.0,3588659.0,3559117.0,0.0,2276287.0,0.0,0.0,0.0,0.0,4779738.0,0.0,0.0,0.0,0.0,31072402.0,0.0,3263200.0,4135188.0,0.0,0.0,0.0,0.0,0.0
2011Q3,29091508.0,6879896.0,18994084.0,4975300.0,31340648.0,0.0,3738000.0,3862067.0,0.0,2416823.0,0.0,0.0,0.0,0.0,4168042.0,0.0,0.0,0.0,0.0,32958620.0,0.0,3292931.0,4462562.0,0.0,0.0,0.0,0.0,0.0
2011Q4,28923606.0,6856120.0,19720172.0,5080475.0,31792171.0,0.0,3937924.0,3849757.0,0.0,2382681.0,0.0,0.0,0.0,0.0,3594393.0,0.0,0.0,0.0,0.0,32571995.0,0.0,3054247.0,4437882.0,0.0,0.0,0.0,0.0,0.0
2012Q1,28063634.0,7061515.0,18561629.0,4955941.0,30116271.0,0.0,3829323.0,3874487.0,0.0,2446994.0,0.0,0.0,0.0,0.0,3457025.0,0.0,0.0,0.0,0.0,29842606.0,0.0,2192619.0,4196191.0,0.0,0.0,0.0,0.0,0.0


# Salvando p/parquet

In [49]:
# dataframe total
# df_trafego.to_parquet(os.path.join(path_dados_derivados,'df_trafego.parquet'))
df_trafego.to_parquet(path_output /'df_trafego.parquet')

In [50]:
# data inicial e final
df_periodos.to_parquet(path_output /'df_periodos.parquet')

In [51]:
# dicionário com os dados de tráfego anuais:
for veq in dict_veqs:
    dict_veqs[veq].to_parquet(path_output / f'df_{veq}_anual.parquet')

NameError: name 'dict_veqs' is not defined

In [ ]:
# dicionário com os dados de tráfego mensial:
for veq in dict_veqs_mensal:
    dict_veqs_mensal[veq].to_parquet(path_output / f'df_{veq}_mensal.parquet')

# Salvando p/Excel

In [ ]:
# dados de tráfego anual
with pd.ExcelWriter(path_output / 'dados_trafego_ANTT_anual.xlsx') as writer:
    df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq in dict_veqs:
        dict_veqs[veq].to_excel(writer, sheet_name=f"{veq}")

In [ ]:
# dados de tráfego mensal
with pd.ExcelWriter(path_output / 'dados_trafego_ANTT_mensal.xlsx') as writer:
    df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq in dict_veqs_mensal:
        dict_veqs_mensal[veq].to_excel(writer, sheet_name=f"{veq}")